In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.losses import CategoricalFocalCrossentropy
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import cv2

tf.keras.mixed_precision.set_global_policy("mixed_float16")
tf.config.optimizer.set_jit(True)

BATCH_SIZE = 128
NUM_SAMPLES_PER_WORD = 500   
WORDS = ['MISS', 'THEY', 'THIS', 'WEST', 'ALSO', 'BEEN', 'DAYS', 'EVEN', 'FILM', 'FROM',
         'GOOD', 'HAVE', 'INTO', 'LAST', 'LIFE', 'LIKE', 'MADE', 'MAKE', 'MANY', 'MORE',
         'MOST', 'MUCH', 'MUST', 'NEED', 'ONLY', 'OVER', 'PART', 'PLAY', 'SAID', 'SOME',
         'SUCH', 'TAKE', 'THAN', 'THAT', 'THEM', 'THEY', 'THIS', 'TIME', 'USED', 'WEEK',
         'WELL', 'WERE', 'WHAT', 'WHEN', 'WILL', 'WITH', 'WORK', 'YEAR', 'YOUR']
WORD_INDEX = {w: i for i, w in enumerate(WORDS)}
NUM_WORDS = len(WORDS)
MAX_LETTERS = max(len(w) for w in WORDS)
IMG_HEIGHT = 28
IMG_WIDTH = IMG_HEIGHT * MAX_LETTERS 

df_train = pd.read_csv("./emnist-byclass-train.csv", header=None)
df_test  = pd.read_csv("./emnist-byclass-test.csv", header=None)

X_train = df_train.drop(columns=[0]).to_numpy()
y_train = df_train[0].to_numpy()
X_test = df_test.drop(columns=[0]).to_numpy()
y_test = df_test[0].to_numpy()

train_mask = (y_train >= 10) & (y_train <= 35)
test_mask  = (y_test >= 10) & (y_test <= 35)

X_train = X_train[train_mask]
y_train = y_train[train_mask] - 10  
X_test = X_test[test_mask]
y_test = y_test[test_mask] - 10

X_train = X_train.reshape(-1, 28, 28)
X_test  = X_test.reshape(-1, 28, 28)

def fix_orientation(images):
    return np.flip(np.rot90(images, k=3, axes=(1, 2)), axis=2)

X_train = fix_orientation(X_train)
X_test = fix_orientation(X_test)

def generate_word(word_list, letter_images, letter_labels, n_samples_per_word):
    X_words, y_words = [], []
    for word in word_list:
        for _ in range(n_samples_per_word):
            imgs = []
            for c in word.upper():
                idx = ord(c) - ord('A')
                candidates = np.where(letter_labels == idx)[0]
                img = letter_images[np.random.choice(candidates)]
                imgs.append(img)
            word_img = np.hstack(imgs)
            word_img = cv2.resize(word_img, (IMG_WIDTH, IMG_HEIGHT))
            X_words.append(word_img[..., np.newaxis].astype(np.float32) / 255.0)
            y_words.append(WORD_INDEX[word.upper()])
    X_words = np.array(X_words)
    y_words = to_categorical(y_words, num_classes=NUM_WORDS)
    return X_words, y_words

X_words, y_words = generate_word(WORDS, X_train, y_train, NUM_SAMPLES_PER_WORD)

X_train, X_val, y_train, y_val = train_test_split(
    X_words, y_words, test_size=0.1, random_state=42, stratify=y_words.argmax(axis=1)
)

augmentation = Sequential([
    layers.RandomRotation(0.05),
    layers.RandomTranslation(0.05, 0.05),
    layers.RandomZoom(0.05),
    layers.RandomContrast(0.05)
])

def prepare_augmentation(x, y):
    x = augmentation(x, training=True)
    return x, y

train_ds = (
    Dataset.from_tensor_slices((X_train, y_train))
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .map(prepare_augmentation, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

val_ds = (
    Dataset.from_tensor_slices((X_val, y_val))
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

2025-09-09 14:00:09.825781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757397609.836699  115472 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757397609.840121  115472 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757397609.849162  115472 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757397609.849173  115472 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757397609.849174  115472 computation_placer.cc:177] computation placer alr

In [2]:
inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 1))

x = layers.Conv2D(16, 3, strides=1, padding="same", use_bias=False)(inputs)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(32, 3, strides=1, padding="same", use_bias=False)(x)

x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(32, 3, strides=2, padding="same", use_bias=False)(x)
x = layers.SpatialDropout2D(0.1)(x)

x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(32, 3, strides=1, padding="same", use_bias=False)(x)

x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(32, 3, strides=1, padding="same", use_bias=False)(x)

x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(64, 3, strides=2, padding="same", use_bias=False)(x)
x = layers.SpatialDropout2D(0.1)(x)

x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(64, 3, strides=1, padding="same", use_bias=False)(x)

x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(64, 3, strides=1, padding="same", use_bias=False)(x)

x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(128, 3, strides=2, padding="same", use_bias=False)(x)
x = layers.SpatialDropout2D(0.15)(x)

x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(128, 3, strides=1, padding="same", use_bias=False)(x)

x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(128, 3, strides=1, padding="same", use_bias=False)(x)

x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.GlobalAveragePooling2D()(x)

x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Dense(128, use_bias=False)(x)

x = layers.Dropout(0.3)(x)

outputs = layers.Dense(NUM_WORDS, activation="softmax", dtype="float32")(x)

model = models.Model(inputs, outputs)

In [3]:
model.compile(
    optimizer=AdamW(learning_rate=1e-3, weight_decay=1e-4),
    loss=CategoricalFocalCrossentropy(gamma=2.0, from_logits=False, label_smoothing=0.1),
    metrics=["accuracy"]
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=[
        ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
        EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
    ]
)

y_val_pred = model.predict(val_ds)
y_val_labels = y_val.argmax(axis=1)
y_pred_labels = y_val_pred.argmax(axis=1)

accuracy = accuracy_score(y_val_labels, y_pred_labels)
print("Validation Accuracy:", accuracy)

Epoch 1/30


I0000 00:00:1757397657.555851  115593 service.cc:152] XLA service 0x708aac008130 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1757397657.555867  115593 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050, Compute Capability 8.6
I0000 00:00:1757397657.563400  115593 cuda_dnn.cc:529] Loaded cuDNN version 91200
I0000 00:00:1757397657.609719  115593 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-09-09 14:00:59.538547: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-09-09 14:01:01.430765: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3733', 4 bytes spill stores, 4 bytes spill loads

2025-09-09 14:01:01.721775: I external/

171/173 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1533 - loss: 0.8310

2025-09-09 14:01:28.944179: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4830', 108 bytes spill stores, 108 bytes spill loads

2025-09-09 14:01:29.275354: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6204', 32 bytes spill stores, 32 bytes spill loads



173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.1558 - loss: 0.8283

2025-09-09 14:01:45.801781: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_29', 44 bytes spill stores, 44 bytes spill loads

2025-09-09 14:01:46.761147: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_380', 28 bytes spill stores, 28 bytes spill loads

2025-09-09 14:01:46.849248: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_380', 68 bytes spill stores, 68 bytes spill loads

2025-09-09 14:01:48.525691: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_380', 8 bytes spill stores, 8 bytes spill loads



173/173 ━━━━━━━━━━━━━━━━━━━━ 55s 155ms/step - accuracy: 0.3720 - loss: 0.6046 - val_accuracy: 0.0273 - val_loss: 0.9928 - learning_rate: 0.0010
Epoch 2/30
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9108 - loss: 0.2098 - val_accuracy: 0.0482 - val_loss: 0.9655 - learning_rate: 0.0010
Epoch 3/30
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9756 - loss: 0.1747 - val_accuracy: 0.8367 - val_loss: 0.3247 - learning_rate: 0.0010
Epoch 4/30
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9867 - loss: 0.1637 - val_accuracy: 0.9967 - val_loss: 0.1508 - learning_rate: 0.0010
Epoch 5/30
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9911 - loss: 0.1581 - val_accuracy: 0.9971 - val_loss: 0.1479 - learning_rate: 0.0010
Epoch 6/30
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9937 - loss: 0.1545 - val_accuracy: 0.9951 - val_loss: 0.1513 - learning_rate: 0.0010
Epoch 7/30
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9942 - loss: 0.1523 - va

In [4]:
tf.saved_model.save(model, "HR")

INFO:tensorflow:Assets written to: HR/assets


INFO:tensorflow:Assets written to: HR/assets


In [ ]:
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model HR tfjs_model

2025-09-09 14:03:42.199695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757397822.211191  118282 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757397822.214669  118282 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757397822.223603  118282 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757397822.223618  118282 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757397822.223622  118282 computation_placer.cc:177] computation placer alr